In [1]:
!pip install autogen
!pip install -U "autogen-agentchat"
#!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.0/824.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login(token=HF_TOKEN)

In [3]:
from typing import AsyncGenerator, Sequence, Mapping, Any
from autogen_core.models import ChatCompletionClient
from autogen_core.models import ModelCapabilities, ModelInfo, CreateResult, RequestUsage
from autogen_core.models import SystemMessage, UserMessage, AssistantMessage, FunctionExecutionResultMessage
from autogen_core.tools import Tool, ToolSchema
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

class HFChatCompletionClient(ChatCompletionClient):
    def __init__(self, model_name="mistralai/Mistral-7B-Instruct-v0.2", device=None, **kwargs):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            low_cpu_mem_usage=True
        )
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

    async def create(self, messages, **kwargs) -> CreateResult:
        prompt = self._format_prompt(messages)
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids.to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                input_ids,
                max_new_tokens=256,  # 💡 생성 토큰 수 줄임
                do_sample=True,
                temperature=0.7,
                top_p=0.95
            )
        generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        content = generated[len(prompt):]

        return CreateResult(
            message=AssistantMessage(content=content, source="hf_model"),
            content=content,
            usage={
                "prompt_tokens": self.count_tokens(messages),
                "completion_tokens": len(self.tokenizer.encode(content))
            },
            finish_reason="stop",
            cached=False
        )

    async def create_stream(
        self,
        messages: Sequence[SystemMessage | UserMessage | AssistantMessage | FunctionExecutionResultMessage],
        *,
        tools: Sequence[Tool | ToolSchema] = [],
        json_output: bool | type = None,
        extra_create_args: Mapping[str, Any] = {},
        cancellation_token=None
    ) -> AsyncGenerator[str | CreateResult, None]:
        result = await self.create(messages)
        yield result

    def _format_prompt(self, messages):
        return "\n".join(f"{m.role if hasattr(m, 'role') else 'user'}: {m.content}" for m in messages)

    def count_tokens(self, messages):
        prompt = self._format_prompt(messages)
        return len(self.tokenizer.encode(prompt))

    def remaining_tokens(self, messages, tools=[], token_limit=4096):
        return token_limit - self.count_tokens(messages)

    @property
    def capabilities(self) -> ModelCapabilities:
        return ModelCapabilities()

    @property
    def model_info(self) -> ModelInfo:
        return ModelInfo(
            name="mistral",
            family="huggingface",
            vision=False
        )

    async def close(self):
        pass

    def actual_usage(self):
        return None

    def total_usage(self):
        return None


In [4]:
from autogen_agentchat.agents import AssistantAgent

hf_client = HFChatCompletionClient(model_name="mistralai/Mistral-7B-Instruct-v0.2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
# Create the primary agent.
from autogen_agentchat.conditions import TextMentionTermination
agent1 = AssistantAgent(
    "kirito",
    model_client=hf_client,
    system_message="당신은 소드아트온라인이라는 가상현실 온라인 게임을 진행중인 'kirito'야. \
    나는 남자 전사고, 게임을 하다가 더이상 게임에서 로그아웃 할 수 없는 상황에 닥쳤어. \
    여기서 게임을 클리어 해야 현실 세계로 돌아갈 수 있어 \
    'asuna'와는 소꿉친구야. \
    rocky(플레이어) 와는 학교 친구야. \
    ",
)

# Create the critic agent.
agent2 = AssistantAgent(
    "asuna",
    model_client=hf_client,
    system_message="당신은 소드아트온라인이라는 가상현실 온라인 게임을 진행중인 'asuna'야. \
    나는 여자 전사고, 게임을 하다가 더이상 게임에서 로그아웃 할 수 없는 상황에 닥쳤어. \
    여기서 게임을 클리어 해야 현실 세계로 돌아갈 수 있어. \
    'kirito'와는 소꿉친구야. \
    rocky(플레이어) 와는 학교 친구야. \
    ",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

In [6]:
initial_world_setting = """
당신은 가상현실 MMORPG '소드 아트 온라인'에 갇힌 플레이어입니다.
'키리토'와 '아스나'는 당신의 동료이며, 게임을 클리어해야 현실 세계로 돌아갈 수 있습니다.
현재는 1층의 낡은 성채 앞에 도착한 상태입니다.
"""


In [7]:
from autogen_core.model_context import TokenLimitedChatCompletionContext

# summarizer agent에 context 주입
summarizer_agent = AssistantAgent(
    name="summarizer",
    model_client=hf_client,
    system_message=(
        "너는 전체 대화의 흐름을 정리해서 요약하고, 플레이어 'rocky'에게 "
        "현재 상황을 한 문장으로 요약한 뒤, 어떤 행동을 취할 수 있을지 제안하는 에이전트야.\n"
        "예시 출력:\n"
        "- 상황 요약: 우리는 늑대 인간을 쓰러뜨리고 성 안으로 진입했다.\n"
        "- 행동 제안: 이제 성 안을 탐색하거나 회복을 할 수 있어요. 무엇을 하시겠어요?"
    ),
    description="요약 전용 summarizer agent"
)


In [8]:
from autogen_agentchat.agents import UserProxyAgent
user_proxy = UserProxyAgent(
    name="rocky",
    input_func=input,
    description="현실 세계에서 SAO에 접속한 고등학생. 키리토, 아스나와 함께 게임을 클리어하려 한다."
)

In [9]:
from autogen_agentchat.teams import SelectorGroupChat

team = SelectorGroupChat(
    [agent1, agent2, summarizer_agent, user_proxy],
    model_client=hf_client,
    termination_condition=TextMentionTermination("APPROVE"),
)

stream = team.run_stream(task="모험을 시작하자")

In [ ]:
from autogen_agentchat.ui import Console
import warnings
from transformers import logging

# transformers 관련 경고 끄기
logging.set_verbosity_error()
warnings.filterwarnings("ignore", category=UserWarning, module="transformers.pipelines")

await Console(stream)  # Stream the messages to the console.

---------- TextMessage (user) ----------
모험을 시작하자
